### Notebook for generating the MnAu2 cell structure inputs for the QE calculations

Following the steps in the bcc-fe-exchange-model notebook, read in an initial .cif file (6-atoms) and 
combine with magnetic patterns. Translates in the x,y direction to increase the number of Mn atoms to allow 
for more magnetic patterns. Configuration files saved to working folder.

- 6 atom configuration allows for NM, FM and AFM-A.
- 12 atom configuration for AFM-A and AFM-G
- 18 atom configuration for additional patters AFM G2, G3

In [17]:
from pathlib import Path
from sys import stdout
import numpy as np
import pandas as pd
from pymatgen import units
from pymatgen.transformations.standard_transformations import SupercellTransformation
import sympy
from scipy.constants import Boltzmann, electron_volt
from sympy import symbols
from sympy.vector import CoordSys3D

from ruamel.yaml import YAML
yaml = YAML()

from neighbormodels.structure import from_file
from neighbormodels.neighbors import count_neighbors
from neighbormodels.interactions import build_model

### 6 atom:

In [46]:
cif_filepath = "mnau2-6.cif"
structure_mnau26 = from_file(structure_file=cif_filepath)
print(structure_mnau26)

Full Formula (Mn2 Au4)
Reduced Formula: MnAu2
abc   :   3.394839   3.394839   8.880636
angles:  90.000000  90.000000  90.000000
Sites (6)
  #  SP      a    b         c
---  ----  ---  ---  --------
  0  Mn    0    0    0
  1  Au    0.5  0.5  0.840319
  2  Au    0.5  0.5  0.159681
  3  Mn    0.5  0.5  0.5
  4  Au    0    0    0.340319
  5  Au    0    0    0.659681


In [49]:
pd.set_option("display.colheader_justify", "left")
pd.set_option("display.html.border", 0)
html_table_style = {"selector": "th", "props": [("text-align", "left")]}

magnetic_patterns = yaml.load(Path("magnetic_patterns-6.yml"))
#pd.DataFrame(magnetic_patterns["6atoms"]).style \
#    .set_table_styles(html_table_style) \
#    .hide_index()

In [50]:
y={}
y['nm'] = magnetic_patterns['6atoms']['nm']
y['f-type'] = magnetic_patterns['6atoms']['f-type']
y['a-type'] = magnetic_patterns['6atoms']['a-type']
new6 = structure_mnau26.copy(y)
new6.make_supercell([5,5,5])
xyz = new6.cart_coords
supercell = pd.concat([pd.DataFrame(new6.species, columns=["species"]),
                          pd.DataFrame(new6.cart_coords, columns=["x","y","z"]), 
                          pd.DataFrame(new6.site_properties["nm"], columns=["nm"]) , 
                          pd.DataFrame(new6.site_properties["f-type"], columns=["f-type"]) ,
                          pd.DataFrame(new6.site_properties["a-type"], columns=["a-type"]) ,
                      ],
                          axis=1, )

supercell.to_csv('mnau2-6.xyz', header=True, index=False, sep='\t')

### 12 Atom:

In [51]:
rotate_cell_45_degrees = SupercellTransformation(
    scaling_matrix=[[1, 1, 0],
                    [1, -1, 0],
                    [0, 0, 1]],
)

structure_mnau212 = rotate_cell_45_degrees.apply_transformation(structure_mnau26)
print(structure_mnau212)

Full Formula (Mn4 Au8)
Reduced Formula: MnAu2
abc   :   4.801027   4.801027   8.880636
angles:  90.000000  90.000000  90.000000
Sites (12)
  #  SP      a    b         c
---  ----  ---  ---  --------
  0  Mn    0    0    0
  1  Mn    0.5  0.5  0
  2  Au    0.5  0    0.840319
  3  Au    1    0.5  0.840319
  4  Au    0.5  0    0.159681
  5  Au    1    0.5  0.159681
  6  Mn    0.5  0    0.5
  7  Mn    1    0.5  0.5
  8  Au    0    0    0.340319
  9  Au    0.5  0.5  0.340319
 10  Au    0    0    0.659681
 11  Au    0.5  0.5  0.659681


In [52]:
structure_mnau212.to(filename="mnau2-12.cif")

In [53]:
pd.set_option("display.colheader_justify", "left")
pd.set_option("display.html.border", 0)
html_table_style = {"selector": "th", "props": [("text-align", "left")]}

magnetic_patterns12 = yaml.load(Path("magnetic_patterns.yml"))
#pd.DataFrame(magnetic_patterns["12atoms"]).style \
#    .set_table_styles(html_table_style) \
#    .hide_index()

In [54]:
x={}
x['nm'] = magnetic_patterns12['12atoms']['nm']
x['f-type'] = magnetic_patterns12['12atoms']['f-type']
x['a-type'] = magnetic_patterns12['12atoms']['a-type']
x['g-type'] = magnetic_patterns12['12atoms']['g-type']
new = structure_mnau2.copy(x)
new.make_supercell([8,8,8])
xyz = new.cart_coords
supercell = pd.concat([pd.DataFrame(new.species, columns=["species"]),
                          pd.DataFrame(new.cart_coords, columns=["x","y","z"]), 
                          pd.DataFrame(new.site_properties["nm"], columns=["nm"]) , 
                          pd.DataFrame(new.site_properties["f-type"], columns=["f-type"]) ,
                          pd.DataFrame(new.site_properties["a-type"], columns=["a-type"]) ,
                          pd.DataFrame(new.site_properties["g-type"], columns=["g-type"]) 
                      ],
                          axis=1, )

supercell.to_csv('mnau2_12.xyz', header=True, index=False, sep='\t')

### 18 Atom: 

In [55]:
structure_mnau218 = structure_mnau2.copy()
structure_mnau218.make_supercell([3,1,1])
print(structure_mnau218)

Full Formula (Mn6 Au12)
Reduced Formula: MnAu2
abc   :  10.184516   3.394839   8.880636
angles:  90.000000  90.000000  90.000000
Sites (18)
  #  SP           a    b         c
---  ----  --------  ---  --------
  0  Mn    0         0    0
  1  Mn    0.333333  0    0
  2  Mn    0.666667  0    0
  3  Au    0.166667  0.5  0.840319
  4  Au    0.5       0.5  0.840319
  5  Au    0.833333  0.5  0.840319
  6  Au    0.166667  0.5  0.159681
  7  Au    0.5       0.5  0.159681
  8  Au    0.833333  0.5  0.159681
  9  Mn    0.166667  0.5  0.5
 10  Mn    0.5       0.5  0.5
 11  Mn    0.833333  0.5  0.5
 12  Au    0         0    0.340319
 13  Au    0.333333  0    0.340319
 14  Au    0.666667  0    0.340319
 15  Au    0         0    0.659681
 16  Au    0.333333  0    0.659681
 17  Au    0.666667  0    0.659681


In [56]:
magnetic_patterns18 = yaml.load(Path("magnetic_patterns-18.yml"))
#pd.DataFrame(magnetic_patterns["18atoms"]).style \
#    .set_table_styles(html_table_style) \
#    .hide_index()

In [57]:
x={}
x['nm'] = magnetic_patterns18['18atoms']['nm']
x['g2-type'] = magnetic_patterns18['18atoms']['g2']
x['g3-type'] = magnetic_patterns18['18atoms']['g3']
x['g4-type'] = magnetic_patterns18['18atoms']['g4']
new = structure_mnau2_mod.copy(x)
new.make_supercell([6,6,6])
xyz = pd.concat([pd.DataFrame(new.species, columns=["species"]),
                          pd.DataFrame(new.cart_coords, columns=["x","y","z"]), 
                          pd.DataFrame(new.site_properties["nm"], columns=["nm"]) , 
                          pd.DataFrame(new.site_properties["g2-type"], columns=["g2-type"]) ,
                          pd.DataFrame(new.site_properties["g3-type"], columns=["g3-type"]) ,
                          pd.DataFrame(new.site_properties["g4-type"], columns=["g4-type"]) ,
                          #pd.DataFrame(new.site_properties["g-type"], columns=["g-type"]) 
                      ],
                          axis=1, )
xyz.to_csv('mnau2_18.xyz', header=True, index=False, sep='\t')

### Calculated energies for different patterns using QE:

In [58]:
qe_energies = pd.read_csv('energy.out', header=None)
qe_energies.columns = ["type", "count", "total_energy", "energy_per_atom", "relative_energy"]

qe_energies

,type,count,total_energy,energy_per_atom,relative_energy
0,nm,6,-3673.331440,-612.221907,0.000000
1,fm,6,-3673.637207,-612.272868,-0.050961
2,a,6,-3673.625594,-612.270932,-0.049026
3,g,12,-7347.227406,-612.268950,-0.047044
4,g2,18,-11020.876017,-612.270890,-0.048983
5,g3,18,-11020.859880,-612.269993,-0.048087


### Getting the neighbor information to calculate the exchange parameters:

In [70]:
kB = 1000 * Boltzmann / electron_volt
mnau2_neighbor_data12 = count_neighbors(cell_structure=structure_mnau212, r=6)
exchange_model12 = build_model(magnetic_patterns=magnetic_patterns12["12atoms"], neighbor_data=mnau2_neighbor_data12)
exchange_model12

,pattern,J1_AuAu,J1_AuMn,J1_MnMn,J2_AuAu,J2_AuMn,J2_MnMn,J3_AuAu,J3_AuMn,J3_MnMn,J4_AuAu,J4_AuMn,J5_AuAu,J5_AuMn,J6_AuAu,J6_AuMn,J7_AuAu,J8_AuAu
0,a-type,0.0,0.666667,0.0,0.0,0.0,0.666667,0.0,-1.333333,-0.666667,0.0,1.333333,0.666667,0.0,-0.666667,0.0,0.0,0.0
1,f-type,0.0,0.666667,0.0,0.0,0.0,0.666667,0.0,1.333333,0.666667,0.0,1.333333,0.666667,0.0,0.666667,0.0,0.0,0.0
2,g-type,0.0,-0.666667,0.0,0.0,0.0,0.666667,0.0,-1.333333,0.666667,0.0,-1.333333,0.666667,0.0,0.666667,0.0,0.0,0.0
3,nm,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0


In [71]:
mnau2_neighbor_data18 = count_neighbors(cell_structure=structure_mnau218, r=6)
exchange_model18 = build_model(magnetic_patterns=magnetic_patterns18["18atoms"], neighbor_data=mnau2_neighbor_data18)
exchange_model18

,pattern,J1_AuAu,J1_AuMn,J1_MnMn,J2_AuAu,J2_AuMn,J2_MnMn,J3_AuAu,J3_AuMn,J3_MnMn,J4_AuAu,J4_AuMn,J5_AuAu,J5_AuMn,J6_AuAu,J6_AuMn,J7_AuAu,J8_AuAu
0,g2,0.0,0.0,1.333333,0.0,0.0,1.333333,0.0,0.0,-2.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,g3,0.0,0.0,0.444444,0.0,0.0,-0.444444,0.0,0.0,-0.888889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,g4,0.0,0.0,1.333333,0.0,0.0,1.333333,0.0,0.0,-2.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,nm,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
